# Training YOLOv8 - Python API

For comparison with launching the training from terminal (`yolo task=detect [...]`)


## Training with AMP

AMP = "Automatic Mixed Precision"

In [ ]:
%pip install -U ultralytics pillow

In [1]:
import os
import time
import sys
import ultralytics
from IPython import display
import torch

display.clear_output()

ultralytics.checks()

Ultralytics YOLOv8.0.196 🚀 Python-3.10.8 torch-2.1.0+cu121 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 5934MiB)
Setup complete ✅ (12 CPUs, 47.0 GB RAM, 123.8/456.9 GB disk)


In [2]:
# Training parameters:
n_epochs = 50
n_patience = 60
batch_size = 4
amp_flag = True

HOME_trunk_segm = os.getcwd()
print("Home: ", HOME_trunk_segm)

torch.backends.cudnn.benchmark = True
# torch.backends.cudnn.enabled = False
torch.backends.cudnn.enabled = True

cudnn_flg = torch.backends.cudnn.is_available()
print("cuDNN available: ", cudnn_flg)
if cudnn_flg:
    print("cuDNN version: ", torch.backends.cudnn.version())

Home:  /home/jovyan/projects/trunk-segmentation
cuDNN available:  True
cuDNN version:  8902


### Model selection

Available YOLOv8 models (from least number of parameters to highest):

* YOLOv8n -> Nano
* YOLOv8s -> Small
* YOLOv8m -> Medium
* YOLOv8l -> Large
* YOLOv8x -> Extra-large

Notice that with bigger models not only the training takes much more time, but also inference does - meaning that, especially on a low-power device as the OAK-D lite, deploying the bigger models may be problematic.

In [3]:
## Select model:
# yolo_model = 'yolov8n.pt'  # Use YOLOv8 'Nano'
# yolo_model = 'yolov8s.pt'  # Use YOLOv8 'Small'
yolo_model = 'yolov8m.pt'  # Use YOLOv8 'Medium'
# yolo_model = 'yolov8l.pt'  # Use YOLOv8 'Large'
# yolo_model = 'yolov8x.pt'  # Use YOLOv8 'Extra-large'

In [4]:
model = ultralytics.YOLO(yolo_model)
t_start = time.time()
results = model.train(data=os.path.join(HOME_trunk_segm, 'data.yaml'), epochs=n_epochs, patience=n_patience, batch=batch_size, imgsz=640, amp=amp_flag)
t_stop = time.time()

New https://pypi.org/project/ultralytics/8.0.203 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.10.8 torch-2.1.0+cu121 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 5934MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/home/jovyan/projects/trunk-segmentation/data.yaml, epochs=50, patience=60, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_widt

In [5]:
print(f"Training time, AMP enabled: {t_stop - t_start} seconds")

Training time, AMP enabled: 12367.697160482407 seconds


### Validation

In [6]:
model.val()

Ultralytics YOLOv8.0.196 🚀 Python-3.10.8 torch-2.1.0+cu121 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 5934MiB)
Model summary (fused): 218 layers, 25840339 parameters, 0 gradients, 78.7 GFLOPs
val: Scanning /home/jovyan/projects/trunk-segmentation/valid/labels.cache... 74 images, 0 backgrounds, 0 corrupt: 100%|██████████| 74/74 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:04<00:00,  4.07it/s]
                   all         74        152       0.66      0.587      0.623      0.346
Speed: 0.6ms preprocess, 54.6ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/val


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f5db0242320>
fitness: 0.37366745713620364
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.34599])
names: {0: 'Tomato trunk'}
plot: True
results_dict: {'metrics/precision(B)': 0.6596933041379824, 'metrics/recall(B)': 0.5866660560602629, 'metrics/mAP50(B)': 0.6227411944189556, 'metrics/mAP50-95(B)': 0.34599259743812005, 'fitness': 0.37366745713620364}
save_dir: PosixPath('runs/detect/val')
speed: {'preprocess': 0.5677004118223448, 'inference': 54.59638544031092, 'loss': 0.003118772764463682, 'postprocess': 0.9473336709512247}

## Train without AMP

In [4]:
HOME = os.getcwd()
print("Home: ", HOME)

model = ultralytics.YOLO('yolov8n.yaml')
t_start_2 = time.time()
results = model.train(data=os.path.join(HOME, 'data.yaml'), epochs=50, patience=200, batch=4, imgsz=640, amp=False)
t_stop_2 = time.time()
# model.val()


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

Home:  /home/jovyan/projects/trunk-segmentation


YOLOv8n summary: 225 layers, 3157200 parameters, 3157184 gradients, 8.9 GFLOPs

New https://pypi.org/project/ultralytics/8.0.199 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.10.8 torch-2.1.0+cu121 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 5934MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/home/jovyan/projects/trunk-segmentation/data.yaml, epochs=50, patience=200, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=False, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=F

In [5]:
print(f"Training time, AMP disabled: {t_stop_2 - t_start_2} seconds")

Training time, AMP disabled: 360.7012538909912 seconds
